In [1]:
!pip install pyathena
from athena_querying import AthenaQuery
from athena_common_queries import *

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
try:
    import psycopg2
except:
    print("Failed ot import psychopg2, trying to install it")
    !{sys.executable} -m pip install psycopg2-binary
    import psycopg2
    print("Successfully installed")
    
    
try:
    import dateparser
except:
    print("Failed ot import dateparser, trying to install it")
    !{sys.executable} -m pip install dateparser
    import dateparser
    print("Successfully installed")
    
import data_pier_querying

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [4]:
from datetime import datetime, timedelta, date

# Settings

In [5]:
num_days_to_query = 5
#from_datetime = datetime.now() - timedelta(days = 5)
from_datetime = datetime(year=2020, month=1, day=10)
to_datetime = from_datetime+ timedelta(days=num_days_to_query)

# Athena (Kinesis) Data

In [12]:
aq = AthenaQuery()

In [13]:
aq.connect()

In [14]:
athena_database = "ms_data_lake_production"
athena_raw_events_table = "ms_data_stream_production_processed"

In [15]:
#query = "select context.page_url, body.event_name, count(*) from "+athena_database+"."+athena_raw_events_table
#query += " where partition_0='2019' and partition_1>='12' and partition_2>='05' group by 1,2"

In [16]:
# I've removed the device_type data to save memory, but it would be useful.
query = create_generic_event_query(from_datetime, to_datetime, include_device_type_data=False, interpret_urls=False)

full_query = "select * from (%s) where country_code ='sg'" %query

In [17]:
print(full_query)

select * from (
    
    SELECT 
          CAST("from_iso8601_timestamp"("sent_at") AS timestamp) "sent_at_timestamp"
    , "sent_at"
    , "type"
    , "body"."event_name"
    , "body"."data"."status"
    , "user"."anonymous_id"
    , "user"."amp_id"
    , "context"."page_url"
    , "context"."referrer"
 
    
    
    FROM
      ms_data_lake_production.ms_data_stream_production_processed
    
    
    WHERE true -- makes query composition easier
    
 AND 
  (
 partition_0 >= '2020'
 AND partition_1 >= '01'
 AND partition_2 >= '10'
 OR (
 partition_0 >= '2020'
 AND partition_1 > '01'
 ) 
 OR (
 partition_0 > '2020'
 ) 
)
 AND ((partition_0 <= '2020'
	 AND partition_1 <= '01'
	 AND partition_2 <= '15'
) 
 OR (
	 partition_0 <= '2020'
	 AND partition_1 < '01'
) 
 OR (
	 partition_0 < '2020'
) 
)
 AND CAST(from_iso8601_timestamp(sent_at) AS timestamp)  between CAST(from_iso8601_timestamp('2020-01-10T00:00:00') AS timestamp) AND CAST(from_iso8601_timestamp('2020-01-15T00:00:00') AS times

In [18]:
athena_full_events_df = aq.query(query)

In [23]:
# Set types to speed queries and save on memory
athena_full_events_df = athena_full_events_df.astype({ "type":"category"
    , "event_name":"category"
    , "status":"category"}, copy=False)

In [24]:
athena_full_events_df.dtypes

sent_at_timestamp      object
sent_at                object
type                 category
event_name           category
status               category
anonymous_id           object
amp_id                 object
page_url               object
referrer               object
dtype: object

In [25]:
athena_full_events_df.head(5)

,sent_at_timestamp,sent_at,type,event_name,status,anonymous_id,amp_id,page_url,referrer
0,2020-01-11 00:42:48.444,2020-01-11T00:42:48.444Z,page,PageView,NaN,c7845715-26f0-4a39-8e06-d37e369ac008,NaN,https://blog.moneysmart.hk/zh-hk/dining/%E5%86...,NaN
1,2020-01-11 00:42:50.202,2020-01-11T00:42:50.202Z,event,UserEngagement.ShowedMoreDetails,NaN,cf37ca17-009d-4ed7-9b98-90a36d9ae239,NaN,https://www.moneysmart.sg/home-loan/compare/loans,https://www.moneysmart.sg/home-loan/compare/re...
2,2020-01-11 00:00:00.632,2020-01-11T00:00:00.632000+00:00,page,PageView,NaN,NaN,amp-vXPuO6NvZYXNPpZ8unTwBg,https://www.moneysmart.id/nih-5-pelajaran-sepu...,NaN
3,2020-01-11 00:42:48.778,2020-01-11T00:42:48.778Z,page,PageView,NaN,de593749-2ad1-4565-93e0-dc62a695e97a,NaN,https://www.moneysmart.sg/embed/3e5d180b4b0b91...,https://s0.2mdn.net/dfp/509788/70424308/157321...
4,2020-01-11 00:42:52.199,2020-01-11T00:42:52.199Z,page,PageView,NaN,2650ffc5-dd18-4e07-81e6-f87e61bb2c62,NaN,https://blog.moneysmart.sg/wedding/wedding-pac...,https://www.google.com/


# Segment Data

NB: screwed up, and can use the tracks table, rather than individual event tables, so a lot of this is pointless.

In [26]:
#from importlib import reload
#reload(data_pier_querying)

In [27]:
dp_querying = data_pier_querying.DataPierQuerying()
dp_querying.connect()

In [28]:
tables_df = dp_querying.query_to_dataframe("select * from information_schema.tables")

In [29]:
segment_event_tables_df = tables_df[tables_df.table_schema=="moneysmartsg_prod"]["table_name"]


In [30]:
# These are taken from the dictionary in https://docs.google.com/spreadsheets/d/1HICh77BoGMIat9K3NPwz3pBayJWiAr0ohAlTuv7dr80/edit#gid=1882048411
#but actually it turns out there should be more than this, and don't need to do it this way.
expected_events_str = """
LeadGeneration.ClickConversion
LeadGeneration.FormStepCompleted
LeadGeneration.FormSubmitted
LeadGeneration.PaymentCompleted
LeadGeneration.ThankYou
LeadGeneration.RedirectCompleted
UserEngagement.ShowedMoreDetails
UserEngagement.ViewedMoreDetails
UserEngagement.SortedList
UserEngagement.UsedHelpHints
UserEngagement.ClickedMenuItem
UserEngagement.QuestionAnswered
UserEngagement.ShowMoreFilter
UserEngagement.ShowMoreOptions
UserEngagement.ClickedFilter
UserEngagement.ButtonClick
UserAuth.LoggedIn
UserAuth.RegisteredAccount
UserAuth.LoggedOut
UserFeedback.ModalDisplayed
UserFeedback.MoodSubmitted
UserFeedback.FeedbackSubmitted
UserFeedback.MoreFeedback
ABTest.Conversion
UserView.WidgetLoad
EmailCapture
PageView
Sharing
Reading
NewsLetterPopup
"""
expected_events = [z.strip() for z in expected_events_str.split("\n") if len(z.strip())>0]

In [31]:


expected_events_and_segment_tables = []
special_maps = {
    "PageView": "pages"
}
for event in expected_events:
    if event in special_maps:
        new_event_name = special_maps[event]
    else:
        new_event_name = ""
        for i, c in enumerate(event):
            if i==0:new_event_name+=c.lower()
            elif str.isupper(c): 
                if i>0 and event[i-1]!=".":
                    new_event_name += "_"
                new_event_name += c.lower()
            elif c==".": new_event_name += "_"
            else: new_event_name+= c
    expected_events_and_segment_tables.append([event, new_event_name])

In [32]:
expected_events_and_segment_tables

[['LeadGeneration.ClickConversion', 'lead_generation_click_conversion'],
 ['LeadGeneration.FormStepCompleted', 'lead_generation_form_step_completed'],
 ['LeadGeneration.FormSubmitted', 'lead_generation_form_submitted'],
 ['LeadGeneration.PaymentCompleted', 'lead_generation_payment_completed'],
 ['LeadGeneration.ThankYou', 'lead_generation_thank_you'],
 ['LeadGeneration.RedirectCompleted', 'lead_generation_redirect_completed'],
 ['UserEngagement.ShowedMoreDetails', 'user_engagement_showed_more_details'],
 ['UserEngagement.ViewedMoreDetails', 'user_engagement_viewed_more_details'],
 ['UserEngagement.SortedList', 'user_engagement_sorted_list'],
 ['UserEngagement.UsedHelpHints', 'user_engagement_used_help_hints'],
 ['UserEngagement.ClickedMenuItem', 'user_engagement_clicked_menu_item'],
 ['UserEngagement.QuestionAnswered', 'user_engagement_question_answered'],
 ['UserEngagement.ShowMoreFilter', 'user_engagement_show_more_filter'],
 ['UserEngagement.ShowMoreOptions', 'user_engagement_show_m

## Check for missing tables

Expect some random events not to be in Segment, or blog specific ones that haven't been deployed to SG and HK

In [33]:
# Check all the event tables exist
expected_event_segment_tables = [z[1] for z in expected_events_and_segment_tables]
segment_table_names = segment_event_tables_df.to_list()
missing_event_tables = [z for z in expected_event_segment_tables if z not in segment_table_names]
missing_event_tables

['user_engagement_used_help_hints',
 'user_engagement_clicked_menu_item',
 'user_feedback_modal_displayed',
 'user_feedback_more_feedback',
 'a_b_test_conversion',
 'sharing',
 'news_letter_popup']

In [34]:
expected_events_and_segment_tables

[['LeadGeneration.ClickConversion', 'lead_generation_click_conversion'],
 ['LeadGeneration.FormStepCompleted', 'lead_generation_form_step_completed'],
 ['LeadGeneration.FormSubmitted', 'lead_generation_form_submitted'],
 ['LeadGeneration.PaymentCompleted', 'lead_generation_payment_completed'],
 ['LeadGeneration.ThankYou', 'lead_generation_thank_you'],
 ['LeadGeneration.RedirectCompleted', 'lead_generation_redirect_completed'],
 ['UserEngagement.ShowedMoreDetails', 'user_engagement_showed_more_details'],
 ['UserEngagement.ViewedMoreDetails', 'user_engagement_viewed_more_details'],
 ['UserEngagement.SortedList', 'user_engagement_sorted_list'],
 ['UserEngagement.UsedHelpHints', 'user_engagement_used_help_hints'],
 ['UserEngagement.ClickedMenuItem', 'user_engagement_clicked_menu_item'],
 ['UserEngagement.QuestionAnswered', 'user_engagement_question_answered'],
 ['UserEngagement.ShowMoreFilter', 'user_engagement_show_more_filter'],
 ['UserEngagement.ShowMoreOptions', 'user_engagement_show_m

In [35]:
# Removing the missing ones from the query list
events_and_tables_to_get_from_data_pier = [z for z in expected_events_and_segment_tables if z[1] not in missing_event_tables]

# Removing a problematic one (doesn't have context_page_url in it, and very unimportant
events_and_tables_to_get_from_data_pier = [z for z in events_and_tables_to_get_from_data_pier if z[1] not in ["user_auth_logged_out",]]

In [36]:
len(events_and_tables_to_get_from_data_pier)

22

In [37]:
cols = dp_querying.query_to_dataframe("""
select column_name, data_type, count(*) from information_schema.columns 
where 
table_name in  ('"""+"','".join([z[1] for z in events_and_tables_to_get_from_data_pier])+"""')
and table_schema='moneysmartsg_prod'

group by 1,2
""")

In [55]:
#cols.sort_values(["count"])

In [39]:
segment_columns_to_query = [
    # "sent_at", - don't use this, use timestamp
    "timestamp",
    #"event", - going to get that implied from the table.
    # "status", # TODO: would like to have this, but not sure which column, or which tables.  Maybe just not used much, so only do for the 4 tables.
    "anonymous_id",
    "context_page_url",
    # "referrer", #maybe only used in pages table??
    "context_ip", 
    "context_user_agent"]

In [40]:
segment_date_constraint = " timestamp >= '%s' and timestamp < '%s' " % (from_datetime.isoformat(), to_datetime.isoformat())

In [41]:
query = "select {cols} from moneysmartsg_prod.{table} where {date_constraint}".format(cols=", ".join(segment_columns_to_query), 
                                                                   table="pages",
                                                                   date_constraint =segment_date_constraint)

In [42]:
query

"select timestamp, anonymous_id, context_page_url, context_ip, context_user_agent from moneysmartsg_prod.pages where  timestamp >= '2020-01-10T00:00:00' and timestamp < '2020-01-15T00:00:00' "

In [43]:
pages = dp_querying.query_to_dataframe("select * from  moneysmartsg_prod.pages where  timestamp > '2020-01-08T15:24:00.263926' and timestamp < '2020-01-15T15:24:00.263966' limit 1000")

In [44]:
click_conversions = dp_querying.query_to_dataframe("select * from  moneysmartsg_prod.lead_generation_click_conversion where  timestamp > '2020-01-08T15:24:00.263926' and timestamp < '2020-01-15T15:24:00.263966' limit 1000")

In [45]:
click_conversions.head()

,id,received_at,channel,context_ip,context_library_name,context_library_version,context_page_referrer,context_user_agent,event_text,product_name,...,product_category_attributes_meta_description,product_category_attributes_meta_title,product_category_type,product_category_attributes_name,product_category_attributes_short_description,product_category_attributes_masthead_title,product_category_attributes_description,product_category_attributes_default,product_category_attributes_slug,product_category_attributes_position
0,ajs-28857690310f6448f49069f94d4e48ec,2020-01-09 02:30:19.509000+00:00,car-insurance,54.255.226.249,analytics.js,3.10.1,None,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,LeadGeneration.ClickConversion,Classic,...,None,None,None,None,None,None,None,None,None,None
1,ajs-319c118f2d0762d259ba66fa47f018e9,2020-01-08 15:33:04.210000+00:00,car-insurance,116.87.146.242,analytics.js,3.10.1,https://www.moneysmart.sg/car-insurance/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,LeadGeneration.ClickConversion,FWD Classic,...,None,None,None,None,None,None,None,None,None,None
2,ajs-59d8a1f8e28e7c2838d5c199635d7264,2020-01-08 15:26:45.031000+00:00,None,119.74.42.156,analytics.js,3.10.1,https://search.yahoo.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,LeadGeneration.ClickConversion,Standard Chartered Unlimited Cashback Credit Card,...,None,None,None,None,None,None,None,None,None,None
3,ajs-6ff3941406c7081456e7c0c2b2d1d089,2020-01-08 15:48:38.556000+00:00,None,27.125.181.147,analytics.js,3.10.1,https://www.google.co.jp/,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,LeadGeneration.ClickConversion,DBS Live Fresh Card,...,None,None,None,None,None,None,None,None,None,None
4,ajs-98805c85d50889f85d0f016873bd68d5,2020-01-08 15:58:22.972000+00:00,None,138.75.26.184,analytics.js,3.10.1,https://www.moneysmart.sg/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,LeadGeneration.ClickConversion,ICBC Global Travel MasterCard Credit Card,...,None,None,None,None,None,None,None,None,None,None


In [46]:
d = dp_querying.query_to_dataframe("select * from moneysmartsg_prod.tracks limit 10")

In [57]:
for a in d.columns: print(a)

id
received_at
context_page_url
original_timestamp
context_library_name
context_library_version
context_page_referrer
context_campaign_source
context_ip
user_id
event_text
uuid_ts
context_campaign_content
context_user_agent
event
context_page_path
context_page_search
context_page_title
sent_at
timestamp
anonymous_id
context_campaign_medium
context_campaign_name
context_campaign_term
context_locale


In [48]:
d["event_text"]

0                             Reading
1                             Reading
2                             Reading
3    LeadGeneration.ShowedMoreDetails
4                             Reading
5                             Reading
6                 UserView.WidgetLoad
7                             Reading
8                             Reading
9                             Reading
Name: event_text, dtype: object

In [49]:

for a in click_conversions.keys():
    print(a)

id
received_at
channel
context_ip
context_library_name
context_library_version
context_page_referrer
context_user_agent
event_text
product_name
provider
context_page_title
event_type
is_paid
list_position
product_id
uuid_ts
context_page_path
context_page_url
provider_id
anonymous_id
auth_status
event
original_timestamp
sent_at
timestamp
action
context_page_search
is_recommended
is_sponsored
source_id
source
affiliate_page_type
affiliate_location
affiliate_category
context_campaign_name
context_campaign_medium
context_campaign_source
context_campaign_content
form_name
page_path
widget_type
affliate_widget_type
page_referrer
provider_slug
product_category_slug
country
page_type
product_slug
product
product_category
context_campaign_term
context_locale
product_category_attributes_meta_description
product_category_attributes_meta_title
product_category_type
product_category_attributes_name
product_category_attributes_short_description
product_category_attributes_masthead_title
product_cate

In [50]:
dp_querying.query_to_dataframe("""SELECT
    nmsp_parent.nspname AS parent_schema,
    parent.relname      AS parent,
    nmsp_child.nspname  AS child_schema,
    child.relname       AS child
FROM pg_inherits
    JOIN pg_class parent            ON pg_inherits.inhparent = parent.oid
    JOIN pg_class child             ON pg_inherits.inhrelid   = child.oid
    JOIN pg_namespace nmsp_parent   ON nmsp_parent.oid  = parent.relnamespace
    JOIN pg_namespace nmsp_child    ON nmsp_child.oid   = child.relnamespace
WHERE parent.relname='%s';""")%"pages"

,parent_schema,parent,child_schema,child


In [51]:
pd.get_option("display.max_colwidth", 200)
indexes = dp_querying.query_to_dataframe("""SELECT
    indexname,
    indexdef
FROM
    pg_indexes
WHERE
    tablename = '%s';""" % "pages")

for a in indexes.values:
    print(a)

['pages_pkey'
 'CREATE UNIQUE INDEX pages_pkey ON moneysmarthk_prod.pages USING btree (id)']
['pages_pkey'
 'CREATE UNIQUE INDEX pages_pkey ON moneysmartsg_prod.pages USING btree (id)']
['pages_timestamp_idx'
 'CREATE INDEX pages_timestamp_idx ON moneysmartsg_prod.pages USING btree ("timestamp")']
['pages_pkey'
 'CREATE UNIQUE INDEX pages_pkey ON moneysmarthk_dev.pages USING btree (id)']
['pages_pkey'
 'CREATE UNIQUE INDEX pages_pkey ON moneysmartsg_dev.pages USING btree (id)']


In [ ]:
query_segment_by_table = False #really shouldn't set this to true, didn't realise correct method.  Also need to add country stuff

segment_schemas = ["moneysmartsg_prod", "moneysmarthk_prod"]
# The meat of it
start_time = datetime.now()
event_name_to_rows = {}
if query_segment_by_table:
    for country_schema in segment_schemas:
        for i, (event_name, table_name) in enumerate(events_and_tables_to_get_from_data_pier):
            table_start_time = datetime.now()
            print("querying table %s / %s (%i/%i)" % (table_name, event_name, i+1, len(events_and_tables_to_get_from_data_pier)))
            query = "select {cols} from {schema}.{table} where {date_constraint}".format(cols=", ".join(segment_columns_to_query), 
                                                                           table=table_name,
                                                                           date_constraint =segment_date_constraint, schema=country_schema)

            events = dp_querying.query_to_dataframe(query)
            events["event_name"] = event_name #fills the entire column with the same value
            print("Got %i events"% len(events))
            event_name_to_rows[event_name]=events

            table_download_time = (datetime.now()-table_start_time).total_seconds()
            time_since_start = (datetime.now()-start_time).total_seconds()
            print("It took %.1f seconds to download from the table (%.1f seconds overall)" %(table_download_time, time_since_start))
            print()
            # if i>4:break


        # Merge tables
        segment_combined_df = pd.DataFrame()
        #combined_df = pd.DataFrame(columns=event_name_to_rows["LeadGeneration.ClickConversion"].columns)
        """for event_name, event_df in event_name_to_rows.items():
            print(len(event_df))
            combined_df.append(event_df, ignore_index=True)
            print(len(combined_df))
        #combined_df.astype({"event_name":"category"})
        """

        segment_combined_df = combined_df.append(list(event_name_to_rows.values()))
    
    
else:
    segment_columns_to_query_full = segment_columns_to_query + ["event_text",]
    tables_to_query = ["pages", "tracks"]
    all_event_dfs = []
    segment_combined_df = pd.DataFrame()
    for country_schema in segment_schemas:
        for table_name in tables_to_query:
            table_start_time = datetime.now()
            if table_name!="pages":
                cols_to_query = segment_columns_to_query_full
            else:
                cols_to_query = segment_columns_to_query
            print("querying table %s.%s" % (country_schema, table_name))
            print(cols_to_query)
            query = "select {cols} from {schema}.{table} where {date_constraint}".format(cols=", ".join(cols_to_query), 
                                                                           table=table_name,
                                                                           date_constraint =segment_date_constraint, schema=country_schema)

            events = dp_querying.query_to_dataframe(query)
            
            print("Got %i events"% len(events))
            #all_event_dfs.append(events)
            
            if table_name =="pages":
                events["event_text"] = "PageView" # fills the whole column
            table_download_time = (datetime.now()-table_start_time).total_seconds()
            time_since_start = (datetime.now()-start_time).total_seconds()
            print("merging")
            segment_combined_df = segment_combined_df.append(events)
            print("It took %.1f seconds to download from the table (%.1f seconds overall)" %(table_download_time, time_since_start))
            print()
            
        

querying table moneysmartsg_prod.pages
['timestamp', 'anonymous_id', 'context_page_url', 'context_ip', 'context_user_agent']
Got 806166 events
merging
It took 9.2 seconds to download from the table (9.2 seconds overall)

querying table moneysmartsg_prod.tracks
['timestamp', 'anonymous_id', 'context_page_url', 'context_ip', 'context_user_agent', 'event_text']
Got 1106775 events
merging
It took 314.5 seconds to download from the table (324.2 seconds overall)

querying table moneysmarthk_prod.pages
['timestamp', 'anonymous_id', 'context_page_url', 'context_ip', 'context_user_agent']
Got 193175 events
merging
It took 267.8 seconds to download from the table (592.4 seconds overall)

querying table moneysmarthk_prod.tracks
['timestamp', 'anonymous_id', 'context_page_url', 'context_ip', 'context_user_agent', 'event_text']
Got 263243 events
merging
It took 70.7 seconds to download from the table (663.8 seconds overall)



In [ ]:
if not query_segment_by_table:
    segment_combined_df.rename(columns={"event_text":"event_name"}, inplace=True)

In [ ]:
len(all_event_dfs)

0

In [ ]:
segment_combined_df.rename(columns={"context_page_url":"page_url"}, inplace=True)
segment_combined_df.head(5)

,timestamp,anonymous_id,page_url,context_ip,context_user_agent,event_name
0,2020-01-10 00:00:01.336000+00:00,b843bc94-64dc-4dca-8978-08877ddaf7eb,https://blog.moneysmart.sg/travel/singapore-pu...,183.90.37.6,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,PageView
1,2020-01-10 00:00:01.915000+00:00,b843bc94-64dc-4dca-8978-08877ddaf7eb,https://www.moneysmart.sg/embed/7432e8a28bd976...,183.90.37.6,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,PageView
2,2020-01-10 00:00:02.871000+00:00,0263aa86-16ca-46d3-b7ee-37bb7665f1da,https://blog.moneysmart.sg/transportation/erp-...,183.90.37.137,Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like...,PageView
3,2020-01-10 00:00:03.139000+00:00,0263aa86-16ca-46d3-b7ee-37bb7665f1da,https://www.moneysmart.sg/embed/52174e7b8d839f...,183.90.37.137,Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like...,PageView
4,2020-01-10 00:00:03.523000+00:00,c25624f4-6ccf-4add-8363-226926ca9737,https://blog.moneysmart.sg/fixed-deposits/best...,119.56.100.216,Mozilla/5.0 (Linux; Android 9; SM-N960F) Apple...,PageView


# Merging Segment and Kinesis Events

In [ ]:
# Make names clear e.g. s_...

# Check the timezone / timestamps match
# Athena raw stuff is in UTC, not SG time.  So 2020-01-19T00:04:04.443Z is 8:05am Singapore time.
# whereas Segment is stored with tiemzone at UTC.  So, could convert them all.
# TODO: But it does meant that there's a lot of events coming at the day boundary.

In [ ]:
athena_full_events_df.head(2)

,sent_at_timestamp,sent_at,type,event_name,status,anonymous_id,amp_id,page_url,referrer
0,2020-01-11 00:42:48.444,2020-01-11T00:42:48.444Z,page,PageView,NaN,c7845715-26f0-4a39-8e06-d37e369ac008,NaN,https://blog.moneysmart.hk/zh-hk/dining/%E5%86...,NaN
1,2020-01-11 00:42:50.202,2020-01-11T00:42:50.202Z,event,UserEngagement.ShowedMoreDetails,NaN,cf37ca17-009d-4ed7-9b98-90a36d9ae239,NaN,https://www.moneysmart.sg/home-loan/compare/loans,https://www.moneysmart.sg/home-loan/compare/re...


In [ ]:
athena_full_events_df.dtypes

sent_at_timestamp      object
sent_at                object
type                 category
event_name           category
status               category
anonymous_id           object
amp_id                 object
page_url               object
referrer               object
dtype: object

In [ ]:
segment_combined_df.dtypes

timestamp             datetime64[ns, UTC]
anonymous_id                       object
page_url                           object
context_ip                         object
context_user_agent                 object
event_name                         object
dtype: object

In [ ]:
# Group by columns to get around date inaccuracy issue
cols_to_group_by = ["anonymous_id", "event_name", "page_url", "date"] #, "context_ip", "context_user_agent"] #TODO: add IP address

print("Grouping by %s"% ", ".join(cols_to_group_by))

print("Fixing dates before grouping")
print("... for Segment")
segment_combined_df["date"] = segment_combined_df.apply(lambda row: row.timestamp.date().isoformat(), axis=1) # making this a string
print("... for athena")
athena_full_events_df["date"] = athena_full_events_df.apply(lambda row: row.sent_at[:10], axis=1)
# super-slow,so moving to using strings athena_full_events_df["date"] = athena_full_events_df.apply(lambda row: dateparser.parse(row.sent_at_timestamp).date(), axis=1)  #conversion from string might not be needed in the future; using dateparser as more robust, also slow

#going to reduce the number of columns to make it safer, then can go back and look for user agents etc (can do a mapping of anonymous_id to user_agent for instance.)




Grouping by anonymous_id, event_name, page_url, date
Fixing dates before grouping
... for Segment
... for athena


In [ ]:
print("Setting sensible data types for the columns to group by")
data_type_mappings = {"event_name":"category", "date":"category"}
segment_combined_df.astype(data_type_mappings, copy=False)
athena_full_events_df.astype(data_type_mappings, copy=False)

Setting sensible data types for the columns to group by


,sent_at_timestamp,sent_at,type,event_name,status,anonymous_id,amp_id,page_url,referrer,date
0,2020-01-11 00:42:48.444,2020-01-11T00:42:48.444Z,page,PageView,NaN,c7845715-26f0-4a39-8e06-d37e369ac008,NaN,https://blog.moneysmart.hk/zh-hk/dining/%E5%86...,NaN,2020-01-11
1,2020-01-11 00:42:50.202,2020-01-11T00:42:50.202Z,event,UserEngagement.ShowedMoreDetails,NaN,cf37ca17-009d-4ed7-9b98-90a36d9ae239,NaN,https://www.moneysmart.sg/home-loan/compare/loans,https://www.moneysmart.sg/home-loan/compare/re...,2020-01-11
2,2020-01-11 00:00:00.632,2020-01-11T00:00:00.632000+00:00,page,PageView,NaN,NaN,amp-vXPuO6NvZYXNPpZ8unTwBg,https://www.moneysmart.id/nih-5-pelajaran-sepu...,NaN,2020-01-11
3,2020-01-11 00:42:48.778,2020-01-11T00:42:48.778Z,page,PageView,NaN,de593749-2ad1-4565-93e0-dc62a695e97a,NaN,https://www.moneysmart.sg/embed/3e5d180b4b0b91...,https://s0.2mdn.net/dfp/509788/70424308/157321...,2020-01-11
4,2020-01-11 00:42:52.199,2020-01-11T00:42:52.199Z,page,PageView,NaN,2650ffc5-dd18-4e07-81e6-f87e61bb2c62,NaN,https://blog.moneysmart.sg/wedding/wedding-pac...,https://www.google.com/,2020-01-11
5,2020-01-11 00:42:55.132,2020-01-11T00:42:55.132Z,event,Reading,Article Body 50,64f44cbe-290d-49c6-b5d8-e489341600c7,NaN,https://blog3.moneysmart.sg/budgeting/how-to-w...,https://www.google.com/,2020-01-11
6,2020-01-11 00:42:48.895,2020-01-11T00:42:48.895Z,event,UserView.WidgetLoad,NaN,de593749-2ad1-4565-93e0-dc62a695e97a,NaN,https://www.moneysmart.sg/embed/3e5d180b4b0b91...,https://s0.2mdn.net/dfp/509788/70424308/157321...,2020-01-11
7,2020-01-11 00:42:50.195,2020-01-11T00:42:50.195Z,page,PageView,NaN,d0e1f96c-0a18-4135-822c-a03b74804e85,NaN,https://blog.moneysmart.sg/dining/cheap-mookat...,https://www.google.com.sg/,2020-01-11
8,2020-01-11 00:00:00.638,2020-01-11T00:00:00.638000+00:00,page,PageView,NaN,NaN,amp-vXPuO6NvZYXNPpZ8unTwBg,https://www.moneysmart.id/fakta-ir-sutami-ment...,NaN,2020-01-11
9,2020-01-11 00:00:00.648,2020-01-11T00:00:00.648000+00:00,page,PageView,NaN,NaN,amp-vXPuO6NvZYXNPpZ8unTwBg,https://www.moneysmart.id/6-pilihan-makanan-en...,NaN,2020-01-11


In [ ]:
segment_combined_df.head()[cols_to_group_by]

,anonymous_id,event_name,page_url,date
0,b843bc94-64dc-4dca-8978-08877ddaf7eb,PageView,https://blog.moneysmart.sg/travel/singapore-pu...,2020-01-10
1,b843bc94-64dc-4dca-8978-08877ddaf7eb,PageView,https://www.moneysmart.sg/embed/7432e8a28bd976...,2020-01-10
2,0263aa86-16ca-46d3-b7ee-37bb7665f1da,PageView,https://blog.moneysmart.sg/transportation/erp-...,2020-01-10
3,0263aa86-16ca-46d3-b7ee-37bb7665f1da,PageView,https://www.moneysmart.sg/embed/52174e7b8d839f...,2020-01-10
4,c25624f4-6ccf-4add-8363-226926ca9737,PageView,https://blog.moneysmart.sg/fixed-deposits/best...,2020-01-10


In [ ]:
athena_full_events_df.head()[cols_to_group_by]

,anonymous_id,event_name,page_url,date
0,c7845715-26f0-4a39-8e06-d37e369ac008,PageView,https://blog.moneysmart.hk/zh-hk/dining/%E5%86...,2020-01-11
1,cf37ca17-009d-4ed7-9b98-90a36d9ae239,UserEngagement.ShowedMoreDetails,https://www.moneysmart.sg/home-loan/compare/loans,2020-01-11
2,NaN,PageView,https://www.moneysmart.id/nih-5-pelajaran-sepu...,2020-01-11
3,de593749-2ad1-4565-93e0-dc62a695e97a,PageView,https://www.moneysmart.sg/embed/3e5d180b4b0b91...,2020-01-11
4,2650ffc5-dd18-4e07-81e6-f87e61bb2c62,PageView,https://blog.moneysmart.sg/wedding/wedding-pac...,2020-01-11


In [ ]:
# athena_full_events_df timestamp

print("Grouping by %s"%cols_to_group_by)
segment_grouped_df = segment_combined_df.groupby(cols_to_group_by).size().reset_index(name='s_count') #size preserves nulls, this sets the column to s_count

athena_grouped_df = athena_full_events_df.groupby(cols_to_group_by).size().reset_index(name='k_count')

# segment_combined_df.rename(columns = {"context_ip":"s_context_ip", "context_user_agent":"s_context_user_agent"}) 

Grouping by ['anonymous_id', 'event_name', 'page_url', 'date']


In [ ]:
athena_grouped_df.head()

,anonymous_id,event_name,page_url,date,k_count
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,2
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-14,1
2,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog3.moneysmart.sg/career/5-easy-side...,2020-01-13,1
3,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-12,1
4,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,3


In [ ]:
# Actually join them

# set the column count names

merged_df = segment_grouped_df.merge(athena_grouped_df, how='outer', on=cols_to_group_by )

#Fill in the empty counts with 0s

merged_df["s_count"].fillna(0, inplace=True)
merged_df["k_count"].fillna(0, inplace=True)

In [ ]:
merged_df

,anonymous_id,event_name,page_url,date,s_count,k_count
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,2.0,2.0
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-13,1.0,0.0
2,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-14,1.0,1.0
3,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-12,1.0,1.0
4,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,3.0,3.0
5,00000b54-600a-4de2-8700-fd9885252dca,UserView.WidgetLoad,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-12,1.0,1.0
6,000034a2-e973-4108-b920-0681877d4fc0,PageView,https://blog.moneysmart.sg/budgeting/cheapest-...,2020-01-10,1.0,1.0
7,00006910-c807-47f9-b57e-0b5e50f91ce1,PageView,https://blog.moneysmart.hk/zh-hk/mortgage/%E6%...,2020-01-13,1.0,1.0
8,00006910-c807-47f9-b57e-0b5e50f91ce1,Reading,https://blog.moneysmart.hk/zh-hk/mortgage/%E6%...,2020-01-13,3.0,3.0
9,0000746e-0fc2-4d90-8962-e9d9553419ed,PageView,https://blog.moneysmart.hk/zh-hk/budgeting/%E8...,2020-01-13,1.0,1.0


In [ ]:
merged_df.groupby(["event_name"]).sum()

,s_count,k_count
event_name,,
Display user feedback form,3.0,0.0
EmailCapture,839.0,848.0
LeadGeneration.ClickConversion,1656.0,1889.0
LeadGeneration.ClickedApplyButton,5504.0,5825.0
LeadGeneration.ClickedCTA,6111.0,5429.0
LeadGeneration.ClickedGoToSite,446.0,0.0
LeadGeneration.Conversion,13757.0,12736.0
LeadGeneration.FormStepCompleted,1707.0,1722.0
LeadGeneration.FormSubmitted,8144.0,8306.0


# Add Filtering Metadata

* is url blog / shop / ...
* country

In [ ]:
from urllib.parse import urlparse, parse_qs

In [ ]:
def get_metadata_from_url(url):
    p = urlparse(url.lower())
    
    #urlparse("https://www-new.moneysmart.sg/rabbit/mouse/?a=b")
    #ParseResult(scheme='https', netloc='www-new.moneysmart.sg', path='/rabbit/mouse/', params='', query='a=b', fragment='')
    
    
    nl = p.netloc
    
    page_type = ""
    stripped_path = p.path.strip("/")
    
    #blog (for SG and HK)
    if "moneysmart.tw" in nl or "moneysmart.ph" in nl or "moneysmart.id" in nl or 'blog.moneysmart' in nl or 'blog-new' in nl:
        page_type = "blog"
    
    #LPS
    elif stripped_path.endswith("ms"):
        page_type = "lps"
    
    #interstitial
    elif "iss.moneysmart" in nl or stripped_path.endswith("apply") or stripped_path.endswith("redirect"):
        page_type = "iss"
        
    #unbounce
    elif "get.moneysmart" in nl:
        page_type = "unbounce"
        
    #embed     , "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 5) like '/embed/%' as is_embed
    
    
    #else shop
    else:
        page_type = "shop"
        
        
        
    
    #ab test side , CAST("strpos"("context"."page_url", '://www-new.') AS boolean) OR CAST("strpos"("context"."page_url", '://www3.') AS boolean)  OR CAST("strpos"("context"."page_url", '://blog3.') AS boolean) as "is_test"
    if "www-new." in nl or "www3." in nl or "blog3." in nl:
        ab_test = "test"
    else:
        ab_test = "control"
    
    slug = "/"+stripped_path
    
    if slug.startswith("/en/") or slug.startswith("/zh-hk/"):
        slug_root = "/"+stripped_path.split("/")[1]
    else:
        slug_root = "/"+stripped_path.split("/")[0]
    
    
    
    
    """
     , CASE WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.sg%' THEN 'sg' 
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.hk%' THEN 'hk' 
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.id%' THEN 'id' 
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.ph%' THEN 'ph'
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.tw%' THEN 'tw' 
        ELSE null END as country_code
    """
    
    country_code = ""
    if "moneysmart.sg" in nl:
        country_code = "sg"
    elif "moneysmart.hk" in nl:
        country_code = "hk"
    elif "moneysmart.id" in nl:
        country_code = "id"
    elif "moneysmart.ph" in nl:
        country_code = "ph"
    elif "moneysmart.tw" in nl:
        country_code = "tw"
    elif "moneysmart.com" in nl:
        country_code = "ww" #worldwide
    else:
        country_code = "??"
    
    
    #return {"page_type":page_type, "path":path, "ab_test":ab_test, "country_code":country_code}
    return [page_type, slug, slug_root, ab_test, country_code]
    
    



In [ ]:
get_metadata_from_url("https://www-new.moneysmart.sg/rabbit/headlight/?scary=True")

['shop', '/rabbit/headlight', '/rabbit', 'test', 'sg']

In [ ]:
get_metadata_from_url("https://blog.moneysmart.ph/rabbit/headlight/?scary=True")

['blog', '/rabbit/headlight', '/rabbit', 'control', 'ph']

In [ ]:
get_metadata_from_url("https://blog3.moneysmart.tw")

['blog', '/', '/', 'test', 'tw']

In [ ]:
get_metadata_from_url("https://www.moneysmart.hk/zh-hk/credit-cards/")

['shop', '/zh-hk/credit-cards', '/credit-cards', 'control', 'hk']

In [ ]:
#This is a bit slow
metadata_df = merged_df.apply(lambda x: pd.Series(get_metadata_from_url(x.page_url)), axis=1)#, index=["page_type", "path", "ab_test", "country_code"])

In [ ]:
metadata_df.rename(columns={0:"page_type", 1:"slug", 2:"slug_root", 3:"ab_test", 4:"country_code"}, inplace=True)

In [ ]:
metadata_df.head()

In [ ]:
merged_df_with_meta = pd.concat([merged_df, metadata_df], axis=1)

In [ ]:
# Set some sensible data types to speed it all up
#merged_df_with_meta.astype({"page_type":"category", "slug":"category"})
merged_df_with_meta = merged_df_with_meta.astype({"page_type":"category", "slug":"category", "ab_test":"category", "country_code":"category", "s_count":"int", "k_count":"int"})

In [ ]:
merged_df_with_meta.head()

In [ ]:
merged_df_with_meta[(merged_df_with_meta.s_count>1) & (merged_df_with_meta.k_count>1)].head()

# Top Level Checks

In [ ]:
def group_by_and_show_count_difference(df, group_by_cols, with_styling=True):
    """
    This expects counts in s_count and k_count
    """
    
    grouped = df.groupby(group_by_cols).sum().reset_index()
    
    grouped["k_vs_s_%"] = grouped.apply(lambda row:(999 if row.k_count else 0) if row.s_count==0 else round(((row.k_count - row.s_count)/row.s_count)*100, 1), axis=1 )
    grouped = grouped[(grouped.k_count>0) | (grouped.s_count>0)] # filters out NaNs after grouping
    
    

    return grouped

In [ ]:
def colour_grouped_table(df):

        

    def color_how_good(value):
        if isinstance(value, str):
            return
        av = abs(value)
        if av<2:
            c = "green"
        elif value <0:
            c =  "red"

        else:
            c= "blue"

        if av>20:
            return "background-color:rgb(250,200,200)"
        return "color:%s"%c # it's just CSS, so you can do background as well.
    return df.style.applymap(color_how_good , subset=["k_vs_s_%"])


In [ ]:
merged_df_with_meta.head()

## By Country


In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "date"])

In [ ]:
colour_grouped_table(g)

## By Event Type

In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "event_name"])
g.sort_values(["country_code", "event_name"], inplace=True)
colour_grouped_table(g)

In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta[merged_df_with_meta.event_name=="LeadGeneration.ClickedApplyButton"], ["country_code", "slug_root", "event_name"])

In [ ]:
g

## By Top Level Slug

In [ ]:
g = group_by_and_show_count_difference(merged_df_with_meta[merged_df_with_meta.page_type!="blog"], ["country_code", "slug_root", "event_name"])
#g.sort_values(["country_code", "slug_root", "event_name"])

pv = pd.pivot_table(g, index=["country_code", "slug_root"], values=["k_count","s_count","k_vs_s_%"], columns=["event_name"], fill_value="")

#colour_grouped_table(pv)
#TODO: not showing up the s_count and k_count :(
pv = pv.swaplevel(0, 1, axis=1).sort_index(axis=1)

In [ ]:
#pv.columns


In [ ]:
pv

In [ ]:
"%02x" % 255

In [ ]:
pv.to_excel("kinesis_vs_segment.xlsx")

In [ ]:

def highlight_cols(cell):
    #use hex colours, or named ones to ensure excel compatibility on export
    if cell=="":
        return ""
    ci = min(100, int(abs(cell*10)))
    if abs(cell)<=2:
        return "color:green;"
    if cell <0:
        return "background-color:#%02x%02x%02x;" % (255,255-ci,255-ci)
    if cell>0:
        return "background-color:#%02x%02x%02x);" % (255-ci,255-ci,255)
    return "background-color:red;"



#pv[:20].style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, 's-count'])
styled = pv[:20].style.applymap(highlight_cols, subset=pv.columns.get_loc_level('k_vs_s_%', level=1)[0]) #special multi-index on column for colour
#can't get they styling to be happy with colors
#styled.to_excel("kinesis_vs_segment.xlsx", engine='openpyxl') #use special engine for formatting https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

styled

## By Type of Page

# Other Issues to Check For
* duplicates
* skipping "https" from the url (observed as a current issue)
* certain browsers having issues
* users creating a lot of duplicate events (and doing the above analysis using sum vs count)

# Search Area

In [ ]:
import ipywidgets as widgets


In [ ]:
country_codes = merged_df_with_meta.country_code.unique().to_list()

In [ ]:
country_codes

In [ ]:
top_level_slugs = list(merged_df_with_meta[merged_df_with_meta.page_type!="blog"].slug_root.unique())

In [ ]:
top_level_slugs.sort()

In [ ]:
event_types = list(merged_df_with_meta.event_name.unique())
event_types.sort()

In [ ]:
def on_search_button_click(b):
    print("I would be searching")

In [ ]:
anonymous_user_input = widgets.Text()
country_code_dropdown = widgets.Select(
    options= country_codes,
    #value='OSX',
    # rows=10,
    description='Country',
    disabled=False
)
search_button = widgets.Button(description='Search', on_click=on_search_button_click)

top_level_slug_select = widgets.Select(options = top_level_slugs, description="slug")
event_type_select = widgets.Select(options=event_types, description="Event")


search_bar = widgets.HBox([anonymous_user_input, country_code_dropdown , top_level_slug_select, event_type_select, search_button])

display(search_bar)

In [ ]:
display(search_button)

In [ ]:
arrays = [np.hstack([['One']*2, ['Two']*2]) , ['A', 'B', 'C', 'D']]
columns = pd.MultiIndex.from_arrays(arrays)
data =  pd.DataFrame(np.random.randn(5, 4), columns=list('ABCD'))
data.columns = columns 

cm = sns.light_palette("green", as_cmap=True)


In [ ]:
data

In [ ]:
data.loc[pd.IndexSlice[:, pd.IndexSlice[:, 'A']]]

In [ ]:
data.columns

In [ ]:
data.style.background_gradient(cmap=cm, subset=data.columns.get_loc_level('A', level=1)[0])
